In [13]:
import geocoder
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import requests
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Wiki data obtained from the previous notebook


In [14]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser")
wiki_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    if len(col)==3:
        PostalCode =col[0].text.replace("\n","")
        Borough = col[1].text.replace("\n","")
        Neighborhood = col[2].text.replace("\n","")  
        if Borough == "Not assigned":
            continue
        if Neighborhood == "Not assigned":
            Neighborhood = Borough            
        wiki_data = wiki_data.append({"PostalCode":PostalCode, "Borough": Borough, "Neighborhood":Neighborhood}, ignore_index=True)


# Use CSV file put in github to extract longitude and latitude

In [15]:
url = "https://raw.githubusercontent.com/AhmcyrCourse/Coursera_Capstone/master/Geospatial_Coordinates.csv"

In [16]:
data_lonlat = pd.read_csv(url)

In [17]:
data_lonlat.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


# Make sure that we extract the longitude and the latitude of the postal code in wiki_data

In [18]:
latitude, longitude =[], []
for l in wiki_data["PostalCode"]:
    for i,j in enumerate(data_lonlat["Postal Code"]):
        if l==j:
            latitude.append(data_lonlat["Latitude"][i]) 
            longitude.append(data_lonlat["Longitude"][i]) 

wiki_data["Latitude"] = latitude
wiki_data["Longitude"] = longitude

In [19]:
wiki_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Explore and cluster the neighborhoods in Toronto and New York

Select Only boroughs that contain the word Toronto

In [20]:
wiki_Toronto= wiki_data[wiki_data['Borough'].apply(lambda x: "Toronto" in x.split())]
wiki_Toronto.reset_index(drop=True, inplace=True)
wiki_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Select Only boroughs that contain the word Toronto

In [21]:
wiki_NY= wiki_data[wiki_data['Borough'].apply(lambda x: "York" in x.split())]
wiki_NY.reset_index(drop=True, inplace=True)
wiki_NY.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


#### Create a map of Toronto with neighborhoods superimposed on top.

In [22]:
# add markers to map
map_Toronto = folium.Map(location=[wiki_Toronto['Latitude'][1], wiki_Toronto['Longitude'][1]], zoom_start=11)
for lat, lng, neighborhood in zip(wiki_Toronto['Latitude'], wiki_Toronto['Longitude'], wiki_Toronto['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

As we did with all of Toronto, let's visualizate New York the neighborhoods in it

In [23]:
# add markers to map
map_NY = folium.Map(location=[wiki_NY['Latitude'][1], wiki_NY['Longitude'][1]], zoom_start=11)
for lat, lng, neighborhood in zip(wiki_NY['Latitude'], wiki_NY['Longitude'], wiki_NY['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY